In [2]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit import transpile
# from qiskit.visulaization import plot_histogram

In [15]:
def counts(circuit,shot=1024):
    simulator=AerSimulator()
    compiled_circuit=transpile(circuit,simulator)
    job=simulator.run(compiled_circuit,shots=shot)
    result=job.result()
    counts=result.get_counts(circuit)
    return counts

In [28]:
def phase_kickback(op,target_state):
    qc=QuantumCircuit(2,1)
    qc.h(0)
    
    # Preparing target bits in eigenstate
    if target_state=='1':
        qc.x(1)
    elif target_state!='0':
        raise ValueError("Target must be 0 or 1")
    qc.barrier()
    
    # Controlled-U
    if op.lower()=='z':
        qc.cz(0,1)
    elif op.lower()=='id':
        identity_qc = QuantumCircuit(1, name='id')
        identity_instruction = identity_qc.to_instruction()
        qc.append(identity_instruction, [1])
        qc.cx(0, 1)
    else:
        raise ValueError("Op must be z or id")
    qc.barrier()
    
    # Hadamard gate to control bit
    qc.h(0)
    qc.measure(0,0)
    print(qc)
    return qc

In [29]:
circuit_z=phase_kickback('z','0')
countsz=counts(circuit_z)
print("Result Z-gate for 0 : ",countsz)

     ┌───┐ ░     ░ ┌───┐┌─┐
q_0: ┤ H ├─░──■──░─┤ H ├┤M├
     └───┘ ░  │  ░ └───┘└╥┘
q_1: ──────░──■──░───────╫─
           ░     ░       ║ 
c: 1/════════════════════╩═
                         0 
Result Z-gate for 0 :  {'0': 1024}


In [30]:
circuit_z=phase_kickback('z','1')
countsz=counts(circuit_z)
print("Result Z-gate for: ",countsz)

     ┌───┐ ░     ░ ┌───┐┌─┐
q_0: ┤ H ├─░──■──░─┤ H ├┤M├
     ├───┤ ░  │  ░ └───┘└╥┘
q_1: ┤ X ├─░──■──░───────╫─
     └───┘ ░     ░       ║ 
c: 1/════════════════════╩═
                         0 
Result Z-gate for:  {'1': 1024}


In [31]:
circuit_id=phase_kickback('id','0')
countsid=counts(circuit_id)
print("Result ID-gate for 0: ",countsid)

     ┌───┐ ░             ░ ┌───┐┌─┐
q_0: ┤ H ├─░─────────■───░─┤ H ├┤M├
     └───┘ ░ ┌────┐┌─┴─┐ ░ └───┘└╥┘
q_1: ──────░─┤ id ├┤ X ├─░───────╫─
           ░ └────┘└───┘ ░       ║ 
c: 1/════════════════════════════╩═
                                 0 
Result ID-gate for 0:  {'0': 503, '1': 521}


In [32]:
circuit_id=phase_kickback('id','1')
countsid=counts(circuit_id)
print("Result ID-gate for 1: ",countsid)

     ┌───┐ ░             ░ ┌───┐┌─┐
q_0: ┤ H ├─░─────────■───░─┤ H ├┤M├
     ├───┤ ░ ┌────┐┌─┴─┐ ░ └───┘└╥┘
q_1: ┤ X ├─░─┤ id ├┤ X ├─░───────╫─
     └───┘ ░ └────┘└───┘ ░       ║ 
c: 1/════════════════════════════╩═
                                 0 
Result ID-gate for 1:  {'1': 505, '0': 519}
